# Make text files with PyMol commands
Make text files with commands that can be run in PyMol to read in PDB files with reassigned b-factors according to antibody escape.

This is specifically designed for the Barnes et al. structures of S2P trimer with Fab, but I think I have set up the config such that it could work for RBD-Fab alone.

In [1]:
import os
from IPython.display import display, HTML
import pandas as pd
import yaml

Read in configuration and PSE config:

In [2]:
with open('../config.yaml') as f:
    config = yaml.safe_load(f)
    
print(f"Reading PSE specs from {config['pse_config']}")
with open(config['pse_config']) as f:
    pse_config = yaml.safe_load(f)

Reading PSE specs from ../data/pse_config.yaml


Make output directory

In [3]:
os.makedirs(config['pse_dir'], exist_ok=True)

Define some global parameters:

In [4]:
outprefix = 'pymol_commands' # make a separate text file for each

# all maps should also be aligned to 6m0j
ACE2_pdb = '6m0j'

# view1
view1 = """\nset_view (\
     0.833226204,   -0.528404295,    0.162850752,\
    -0.477664202,   -0.539514899,    0.693361998,\
    -0.278514773,   -0.655512929,   -0.701937377,\
     0.000000000,   -0.000000000, -286.698059082,\
   -23.955575943,   16.451217651,    0.807323456,\
  -15130.423828125, 15703.833984375,  -20.000000000 )"""

# which set of data to show, and how to color surfaces.
pymol_specs = {
        'metric' : 'max',
        'color_min' : 'white',
        'color_max' : 'red',
        'view' : view1,
        }

Write function to perform write the same generic commands to text file for every antibody:

In [5]:
def initialize_commands():
    text = """
#commands to load pdbs

reinitialize
set seq_view, 0
set ray_shadows, 0
set spec_reflect, 1
set spec_power, 100000
set sphere_scale, 0.75

load 6m0j_b-factor-mean-bind.pdb
load 6m0j_b-factor-mean-expr.pdb

# create ACE2, RBD_bind, RBD_expr
# hide all
create ACE2, 6m0j_b-factor-mean-bind and chain A
create RBD_bind, 6m0j_b-factor-mean-bind and chain E; remove RBD_bind and chain A
create RBD_expr, 6m0j_b-factor-mean-expr and chain E; remove RBD_expr and chain A
delete 6m0j_b-factor-mean-bind
delete 6m0j_b-factor-mean-expr

# start showing stuff
show cartoon, ACE2; color gray20, ACE2; set cartoon_transparency, 0.5, ACE2
show cartoon, RBD_bind; spectrum b, red white, RBD_bind, minimum=-2, maximum=0; show sticks, RBD_bind and resn NAG
show cartoon, RBD_expr; spectrum b, red white, RBD_expr, minimum=-2, maximum=0; show sticks, RBD_expr and resn NAG

""" 
    f.write(text)

Write function to align trimer-Fab structure to 6m0j and to create Fab, RBD, and spike chains

Might be able to use formatting like this [example](https://stackoverflow.com/questions/16162383/how-to-easily-write-a-multi-line-file-with-variables-python-2-6/16162599).

In [6]:
def load_trimer_fab(ab, sln, pdb, metric, RBD_chains, heavy_chains, light_chains, trimer):

    f.write('# load Fab-trimer PDB and align structures\n')
    f.write(f'load data/{ab}_{sln}_{pdb}_{metric}_escape.pdb\n')
    f.write(f'set_name {ab}_{sln}_{pdb}_{metric}_escape, {pdb}\n')
    f.write(f'align {pdb} and (chain {RBD_chains[0]} and resi 333-526), RBD_bind\n\n')

    f.write('# create Fab chains\n')
    f.write(f'show_as cartoon, {pdb}; color deepblue, {pdb};\n')
    
    for hc, lc in zip(heavy_chains, light_chains):
        f.write(f'create {hc}, {pdb} and chain {hc}; create {lc}, {pdb} and chain {lc}\n')
    
    hc_string = ' or '.join(heavy_chains)
    lc_string = ' or '.join(light_chains)
    f.write(f'color deepblue, ({hc_string}); color marine, ({lc_string})\n\n')

    f.write('# create RBD chains\n')
    for rbd in RBD_chains:
        f.write(f'create RBD_{rbd}, {pdb} and chain {rbd} and resi 333-526; show cartoon, RBD_{rbd}; show sticks, RBD_{rbd} and resn NAG; spectrum b, white red, RBD_{rbd}, minimum=0\n')
    
    if trimer:
        spikes = ['spike_{0}'.format(rbd) for rbd in RBD_chains]
        spike_string = ' or '.join(spikes)
        f.write('\n# create non-RBD spike chains\n')
        for rbd in RBD_chains:
            f.write(f'create spike_{rbd}, {pdb} and chain {rbd} and resi 1-332+527-1253\n')
        f.write(f'color slate, ({spike_string})\n\n')

Write function to show ACE2 contact sites as mesh

In [7]:
def ace2_contact(RBD_chains):
    rbds = ['RBD_{0}'.format(rbd) for rbd in RBD_chains]
    rbd_string = ' or '.join(rbds)
    
    f.write('# show ACE2 contact sites as mesh\n')
    f.write(f'create ACE2_contacts, ({rbd_string}) and resi 417+446+449+453+455+456+475+486+487+489+493+496+498+500+501+502+505\n')
    f.write(f'show mesh, ACE2_contacts; color gray40, ACE2_contacts\n')
    f.write(f'sele ACE2_cont, ({rbd_string}) and resi 417+446+449+453+455+456+475+486+487+489+493+496+498+500+501+502+505\n\n')

Write function to create antibody contact sites

In [8]:
def fab_contacts(RBD_chains, rbd_fab, all_ct_sites):
    """
    rbd_fab is a dictionary keyed by RBD chain (eg A, B, C)
        values are a list of the contact sites for that chain
    all_ct_sites is a list of sites that contact any RBD chain 
    """
    rbds = ['RBD_{0}'.format(rbd) for rbd in RBD_chains]
    rbd_string = ' or '.join(rbds)
    all_contacts = '+'.join(all_ct_sites)
    
    f.write('# create antibody contact sites and show as spheres\n')
    for rbd, sites in rbd_fab.items():
        rbd_contacts = '+'.join(sites)
        f.write(f'sele RBD_{rbd}_ct, RBD_{rbd} and resi {rbd_contacts}\n')
        f.write(f'show spheres, RBD_{rbd}_ct and name ca\n')
    f.write(f'sele all_ct, ({rbd_string}) and resi {all_contacts}\n')
    f.write(f'show spheres, all_ct and name ca\n\n')

Read in contact sites

In [9]:
contacts = pd.read_csv(os.path.join('../', config['structural_contacts']))
contacts['position'] = contacts['position'].astype(str)
contacts.head()

,name,pdb,chain,position
0,ACE2,6M0J,E,417
1,ACE2,6M0J,E,446
2,ACE2,6M0J,E,449
3,ACE2,6M0J,E,453
4,ACE2,6M0J,E,455


Loop through all the PDBs in the config and write output file.

In [10]:
for name, specs in pse_config.items():
    pdb = name
    ab = specs["antibody"]
    sln = specs["selection"]
    RBD_chains = specs["RBD_chains"]
    heavy_chains = specs["heavy_chains"]
    light_chains = specs["light_chains"]
    trimer = specs["trimer"]
    
    # rbd_fab is a dict, keyed by RBD chain; values are list of the contact sites
    rbd_fab = ((contacts
                .query('name==@ab & pdb==@pdb'))
               [['chain', 'position']]
               .reset_index(drop=True)
               .groupby('chain')['position'].apply(list).to_dict()
              )
    
    # create list of all contact sites for a given antibody
    all_ct = []
    for v in rbd_fab.values():
        all_ct.extend(v)
    all_ct_sites = sorted(list(set(all_ct))) 
    
    metric = pymol_specs['metric']
    
    outFile = os.path.join(config['pse_dir'], f'pymol_{ab}_{name}.txt')
    print(outFile)
    
    f = open(outFile, "w")
    f.write(f"# PyMol commands for {ab} {name}")
    initialize_commands()
    load_trimer_fab(ab, sln, pdb, metric, RBD_chains, heavy_chains, light_chains, trimer)
    ace2_contact(RBD_chains)
    fab_contacts(RBD_chains, rbd_fab, all_ct_sites)
    
    f.write("""
#view1
set_view (\
     0.833226204,   -0.528404295,    0.162850752,\
    -0.477664202,   -0.539514899,    0.693361998,\
    -0.278514773,   -0.655512929,   -0.701937377,\
     0.000000000,   -0.000000000, -286.698059082,\
   -23.955575943,   16.451217651,    0.807323456,\
  -15130.423828125, 15703.833984375,  -20.000000000 )
  
""")
    f.write(f'save {ab}_{pdb}.pse\n')
    f.write(f'# png {ab}_{pdb}_view1.png, ray=1, 600, 600')
    
    f.close()

../results/pymol_commands/pymol_C105_6xcn.txt
../results/pymol_commands/pymol_C105_6xcm.txt
../results/pymol_commands/pymol_C144_7k90.txt
../results/pymol_commands/pymol_C002_7k8s.txt
../results/pymol_commands/pymol_C002_7k8t.txt
../results/pymol_commands/pymol_C121_7k8x.txt
../results/pymol_commands/pymol_C121_7k8y.txt
../results/pymol_commands/pymol_C135_7k8z.txt
../results/pymol_commands/pymol_C110_7k8v.txt
